# Notebook fuer Seebruecke

In [52]:
import csv
import os
import json
import pandas as pd
import seaborn as sns
import urllib
import urllib.request
import matplotlib.pyplot as plt
import logging
import urllib.request
import chimpy
import hashlib
from environs import Env

In [2]:
env = Env()
env.read_env()  

mc_api_key = env("MAILCHIMP_API_KEY") 

## Problem 1: Automatisiere Mail export aus FundraisingBox und Import nach Mailchimp

In [3]:
df_fundraising_box = pd.read_csv("FundraisingBox_sample_Andreas.csv", delimiter=";")
df_mailchimp = pd.read_csv("sample-mailchimp-file-import-CSV_-_sample-mailchimp-import-file__1_.csv.csv"
                           , delimiter=",")

### Analyse

In [4]:
pd.set_option('display.max_columns', None)
df_fundraising_box.head(20)

,donation_id,ident_id,external_donation_id,transaction_id,status,donated_at,donated_at_date,donated_at_time,received_at,received_at_date,received_at_time,amount,interval,by_recurring,recurring_id,source,source_id,source_name,source_promotion_code,type,type_id,type_promotion_code,project,project_id,project_promotion_code,receipt_status,donation_info,donation_meta_info,donation_meta_info_readable,device_type,custom1,custom2,custom3,bank_account_owner,bank_account_number,bank_number,bank_iban,bank_bic,bank_name,bank_country,credit_card_owner,credit_card_number_hidden,credit_card_number_expire_date_month,credit_card_number_expire_date_year,sepa_mandate_bank_account_owner,sepa_mandate_iban,sepa_mandate_bic,sepa_mandate_reference_id,sepa_mandate_status,sepa_mandate_link,sepa_mandate_signature_date,sepa_mandate_transaction_type,public_name,public_message,fundraising_page_id,fundraising_page_title,payout_id,payout_at,external_payment_fee,external_payment_fee_currency,person_id,external_person_id,salutation,title,first_name,last_name,company_name,position,employee_of,employee_company_address,employee_company_post_code,employee_company_city,employee_company_state,employee_company_country,wants_mailing,wants_newsletter,wants_no_email,wants_no_post,wants_no_call,greeting,person_info,birthday,person_tags,smart_contact_id,gift_aid,person_created_at,donation_sum,donation_sum_last_12_months,address,post_code,city,state,country,email_address,phone,Konto-Eingang,Kontoauszüge-Ordner,Lokalgruppe,trx_id Twingle,twingle donation finished,Twingle Papypal Charity Fees,Twingle Payment Method,Twingle Payout Reference,Twingle purpose,Uhrzeit,Uhrzeit Überweisung
0,12318523,FB-29387,NaN,FB-T-9560638,confirmed,2021-04-11 04:46:17,2021-04-11,04:46:17,NaN,NaN,NaN,2.0,1.0,1.0,477831.0,payment_form,3,Seebrücke Spende,NaN,wikando_direct_debit,15,NaN,Seebrücke,44456,NaN,no_receipt,NaN,NaN,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sally Meier,DE12341234123412341234,NaN,FB-SEPA-8326504,active,https://secure.fundraisingbox.com/mandates/dow...,2020-12-11,RCUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7922610,NaN,Mrs.,NaN,X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-11,10.0,10.0,NaN,NaN,NaN,NaN,NaN,mensch@mustermail.de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12318515,FB-29386,NaN,FB-T-9560630,confirmed,2021-04-11 04:46:10,2021-04-11,04:46:10,NaN,NaN,NaN,15.0,1.0,1.0,477748.0,payment_form,3,Seebrücke fördern Newsletter,NaN,wikando_direct_debit,15,NaN,Seebrücke fördern,35882,NaN,receipt_end_of_year,NaN,NaN,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heidi Müller,DE12341234123412341234,NaN,FB-SEPA-8325743,active,https://secure.fundraisingbox.com/mandates/dow...,2020-12-11,RCUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7921437,NaN,Mrs.,NaN,X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-11,75.0,75.0,Musterstraße 1,12345.0,Musterort,NaN,DE,mensch@mustermail.de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12318399,FB-29385,NaN,FB-T-9560514,confirmed,2021-04-11 04:45:12,2021-04-11,04:45:12,NaN,NaN,NaN,20.0,1.0,1.0,477489.0,payment_form,3,Seebrücke fördern Newsletter,NaN,wikando_direct_debit,15,NaN,Seebrücke fördern,35882,NaN,receipt_end_of_year,NaN,NaN,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sebastian und Martha Rosenfeld,DE12341234123412341234,NaN,FB-SEPA-8322407,active,https://secure.fundraisingbox.com/mandates/dow...,2020-12-11,RCUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7916196,NaN,Mr.,NaN,X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-11,100.0,100.0,Musterstraße 1,12345.0,Musterort,NaN,DE,mensch@mustermail.de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12315416,FB-29384,NaN,FB-T-9557530,confirmed,2021-04-11 01:55:45,2021-04-11,01:55:45,NaN,NaN,NaN,10.0,1.0,0.0,541362.0,payment_form,3,Seebrücke Spende,NaN,wikando_direct_debit,15,NaN,Seebrücke,44456,NaN,receipt_end_of_year,NaN,"{""wants_newsletter"":"""",""i

In [5]:
print(df_fundraising_box.columns)

Index(['donation_id', 'ident_id', 'external_donation_id', 'transaction_id',
       'status', 'donated_at', 'donated_at_date', 'donated_at_time',
       'received_at', 'received_at_date',
       ...
       'Kontoauszüge-Ordner', 'Lokalgruppe', 'trx_id Twingle',
       'twingle donation finished', 'Twingle Papypal Charity Fees',
       'Twingle Payment Method', 'Twingle Payout Reference', 'Twingle purpose',
       'Uhrzeit', 'Uhrzeit Überweisung'],
      dtype='object', length=106)


In [6]:
df_mailchimp.head()

,Email Address,First Name,Last Name,Address,Phone,What do you like to grow?,Tags,Birthday
0,m.white@student.hathaway.edu,Maya,White,154 Example Dr. Hendersonville NC 28792 US,(555) 555-5555,Indoor plants,"influencer, uses coupons, local",08/03
1,b.mwangi@hathaway.edu,Benjamin,Mwangi,7553 Example St. Apt. 234 Bonita Springs FL...,(555) 222-3333,"Tropical plants, Fruit trees","trade show, influencer",12/29
2,e.nemec@hathaway.edu,Elizabeth,Nemec,221 Example Lane Toronto ON M5G 1M6 CA,(555) 111-4444,"Herbs, Fruit trees",local,07/28
3,=(^.^)=PUGOVKA=(^.^)=,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ut subscription,you can only download one file at a time...,NaN,NaN,NaN,NaN,NaN,NaN


### Transformiere Datei aus der FundraisingBox

Get rid of entries with no donation meta info

In [7]:
df_fund_trans = df_fundraising_box.dropna(subset=['donation_meta_info']).copy()

Fill missing values with an empty string

In [8]:
df_fund_trans = df_fund_trans.fillna('')

Extract information about if the user wants a newsletter

In [9]:
df_fund_trans['donation_meta_info_dict'] = df_fund_trans.apply(lambda x: json.loads(x['donation_meta_info']), axis=1)
df_fund_trans['wants_nl'] = df_fund_trans.apply(lambda x: json.loads(x['donation_meta_info'])['wants_newsletter'], axis=1)

Select all donators who want a newsletter

In [10]:
df_fund_want_nl = df_fund_trans[df_fund_trans['wants_nl'] == '1'].copy()

Cast some datetypes

In [11]:
df_fund_want_nl['post_code'] = df_fund_want_nl['post_code'].astype(int)
df_fund_want_nl['post_code'] = df_fund_want_nl['post_code'].astype(str)
df_fund_want_nl['state'] = df_fund_want_nl['state'].astype(str)

Transform columns from FundraisingBox to a Mailchimp compatible format

In [12]:
df_fund_want_nl['address_for_chimp'] = df_fund_want_nl.apply(
    lambda x: x['address']+'  '
    +x['city']+'  '
    +x['state']+'  '
    +x['post_code']+'  '
    +x['country']
    , axis=1)

Find out if donator is recurring

In [13]:
df_fund_want_nl['ist_dauerspender'] = df_fund_want_nl.apply(lambda x: 1 if x['by_recurring']==1 else 0, axis=1)
df_fund_want_nl['ist_einzelspender'] = df_fund_want_nl.apply(lambda x: 0 if x['ist_dauerspender']==1 else 1, axis=1)

Testing

In [14]:
df_fund_want_nl.head()

,donation_id,ident_id,external_donation_id,transaction_id,status,donated_at,donated_at_date,donated_at_time,received_at,received_at_date,received_at_time,amount,interval,by_recurring,recurring_id,source,source_id,source_name,source_promotion_code,type,type_id,type_promotion_code,project,project_id,project_promotion_code,receipt_status,donation_info,donation_meta_info,donation_meta_info_readable,device_type,custom1,custom2,custom3,bank_account_owner,bank_account_number,bank_number,bank_iban,bank_bic,bank_name,bank_country,credit_card_owner,credit_card_number_hidden,credit_card_number_expire_date_month,credit_card_number_expire_date_year,sepa_mandate_bank_account_owner,sepa_mandate_iban,sepa_mandate_bic,sepa_mandate_reference_id,sepa_mandate_status,sepa_mandate_link,sepa_mandate_signature_date,sepa_mandate_transaction_type,public_name,public_message,fundraising_page_id,fundraising_page_title,payout_id,payout_at,external_payment_fee,external_payment_fee_currency,person_id,external_person_id,salutation,title,first_name,last_name,company_name,position,employee_of,employee_company_address,employee_company_post_code,employee_company_city,employee_company_state,employee_company_country,wants_mailing,wants_newsletter,wants_no_email,wants_no_post,wants_no_call,greeting,person_info,birthday,person_tags,smart_contact_id,gift_aid,person_created_at,donation_sum,donation_sum_last_12_months,address,post_code,city,state,country,email_address,phone,Konto-Eingang,Kontoauszüge-Ordner,Lokalgruppe,trx_id Twingle,twingle donation finished,Twingle Papypal Charity Fees,Twingle Payment Method,Twingle Payout Reference,Twingle purpose,Uhrzeit,Uhrzeit Überweisung,donation_meta_info_dict,wants_nl,address_for_chimp,ist_dauerspender,ist_einzelspender
8,12314150,FB-29375,,FB-T-9556135,confirmed,2021-04-10 15:10:59,2021-04-10,15:10:59,,,,10.0,1.0,0.0,541091.0,payment_form,3,Seebrücke Spende,,wikando_direct_debit,15,,Seebrücke,44456,,receipt_end_of_year,,"{""message"":""Danke f\u00fcr alles!"",""wants_news...",message: \nDanke für alles!\nwants_newsletter:...,mobile,,,,,,,,,,,,,,,Tristan Reck,DE12341234123412341234,,FB-SEPA-9556135,active,https://secure.fundraisingbox.com/mandates/dow...,2021-04-10,RCUR,,,,,,,,,9648933,,Mr.,,X,Y,,,,,,,,,1,1,0,0,0,,,,,,,2021-04-10,10.0,10.0,Musterstraße 1,12345,Musterort,,DE,mensch@mustermail.de,,,,,,,,,,,,,"{'message': 'Danke für alles!', 'wants_newslet...",1,Musterstraße 1 Musterort 12345 DE,0,1


In [15]:
for row in df_fund_trans['donation_meta_info_dict']:
    print(row)

{'wants_newsletter': '', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen'}
{'wants_newsletter': '', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen', 'paypal_mc_fee': '0.97'}
{'message': 'Spenden, die wir mit unserer Postkartenaktion gesammelt haben! Wir sind 9 Künstlerinnen aus Augsburg.', 'wants_newsletter': '', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen'}
{'message': 'Zur freien Verwendung für die Lokalgruppe Hörstel', 'wants_newsletter': '', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen', 'paypal_mc_fee': '1.35'}
{'message': 'Danke!', 'wants_newsletter': '', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen'}
{'message': 'Danke für alles!', 'wants_newsletter': '1', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen'}
{'message': 'Danke für eure Arbeit. Grüße von eurer Lieblingsfach. Politikwissenschaft und Soziologie', 'wants_newsletter': '', 'item_id': 34395, 'item_name': 'Seebrücke unterstützen', 'paypal_mc_fee': '2.47'}
{'wants_newslet

In [16]:
# print(df_fund_want_nl['donation_meta_info_json'])

In [17]:
# for row in df_fund_want_nl['donation_meta_info_json']:
#     print(row)

### Bereite Daten fuer Mailchimp vor

In [18]:
cols_for_chimp = ['email_address', 'first_name', 'last_name', 'address_for_chimp'
                  , 'phone', 'donation_id', 'ist_dauerspender', 'ist_einzelspender']

In [19]:
df_for_chimp = df_fund_want_nl[cols_for_chimp].copy()

Reduziere auf einen Eintrag pro e-mail Adresse

In [20]:
df_for_chimp_agg = df_for_chimp.groupby('email_address').agg({
    'donation_id': ['min','max'], 'ist_dauerspender': 'sum', 'ist_einzelspender':'sum'})

In [21]:
df_for_chimp_agg.columns = ["_".join(x) for x in df_for_chimp_agg.columns.ravel()]

<ipython-input-21-8068da92627e>:1: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df_for_chimp_agg.columns = ["_".join(x) for x in df_for_chimp_agg.columns.ravel()]


In [22]:
df_for_chimp_agg = df_for_chimp_agg.reset_index()

In [23]:
df_for_chimp_agg.head()

,email_address,donation_id_min,donation_id_max,ist_dauerspender_sum,ist_einzelspender_sum
0,mensch@mustermail.de,12314150,12314150,0,1


Fuege Tag hinzu, ob Dauerspender, Einzelspender oder beides

In [24]:
df_for_chimp_agg['spender_tag'] = df_for_chimp_agg.apply(lambda x: 
            'Einzelspender/in' if ((x['ist_dauerspender_sum'] == 0) & (x['ist_einzelspender_sum'] > 0)) else
                ('Dauerspender/in' if ((x['ist_dauerspender_sum'] > 0) & (x['ist_einzelspender_sum'] == 0)) else
                    ('Einzelspender/in, Dauerspender/in' if ((x['ist_dauerspender_sum'] > 0) & (x['ist_einzelspender_sum'] > 0)) else '' ) )
            , axis=1)

Testing

In [25]:
df_for_chimp_agg.head()

,email_address,donation_id_min,donation_id_max,ist_dauerspender_sum,ist_einzelspender_sum,spender_tag
0,mensch@mustermail.de,12314150,12314150,0,1,Einzelspender/in


Combine Data

In [26]:
df_for_chimp_out = pd.merge(df_for_chimp_agg, df_for_chimp
                            ,  how='left'
                            , left_on=['email_address','donation_id_max']
                            , right_on = ['email_address','donation_id'])

In [27]:
df_for_chimp_out.head()

,email_address,donation_id_min,donation_id_max,ist_dauerspender_sum,ist_einzelspender_sum,spender_tag,first_name,last_name,address_for_chimp,phone,donation_id,ist_dauerspender,ist_einzelspender
0,mensch@mustermail.de,12314150,12314150,0,1,Einzelspender/in,X,Y,Musterstraße 1 Musterort 12345 DE,,12314150,0,1


### Passe Datei aus FundraisingBox and das Format fuer Mailchimp an

In [28]:
df_mailchimp_wip = df_for_chimp_out.copy()

Extract relevant columns for output

In [29]:
df_mailchimp_wip.columns

Index(['email_address', 'donation_id_min', 'donation_id_max',
       'ist_dauerspender_sum', 'ist_einzelspender_sum', 'spender_tag',
       'first_name', 'last_name', 'address_for_chimp', 'phone', 'donation_id',
       'ist_dauerspender', 'ist_einzelspender'],
      dtype='object')

In [30]:
col_map = [
    ('email_address', 'Email Address'),
    ('first_name', 'First Name'),
    ('last_name', 'Last Name'),
    ('address_for_chimp', 'Address'),
    ('phone', 'Phone'),
#     ('', 'What do you like to grow?'),
    ('spender_tag', 'Tags'),
#     ('', 'Birthday'),
]
output_cols = []

In [31]:
output_cols = []
for mapping in col_map:
    col_fund = mapping[0]
    col_chimp = mapping[1]
    output_cols.append(col_chimp)
    df_mailchimp_wip[col_chimp] = df_mailchimp_wip[col_fund]

In [32]:
df_mailchimp_output = df_mailchimp_wip[output_cols].copy()

In [33]:
df_mailchimp_output.head()

,Email Address,First Name,Last Name,Address,Phone,Tags
0,mensch@mustermail.de,X,Y,Musterstraße 1 Musterort 12345 DE,,Einzelspender/in


Schreibe Ergebnis als csv

In [34]:
fname = 'import_into_mailchimp.csv'
df_mailchimp_output.to_csv(fname,sep=',', index=False)

## Schreibe Ergebnisse nach mailchimp

In [35]:
def test_ping():
    assert chimp.ping() == "Everything's Chimpy!"

In [36]:
def setup_module(api_key):
#     assert 'MAILCHIMP_APIKEY' in os.environ, \
#         "please set the MAILCHIMP_APIKEY environment variable\n" \
#         "you can get a new api key by calling:\n" \
#         " wget 'http://api.mailchimp.com/1.1/?output=json&method=login" \
#         "&password=xxxxxx&username=yyyyyyyy' -O apikey"


    global chimp
    chimp = chimpy.Connection(api_key)

In [37]:
def test_lists():
    lists = chimp.lists()
    pprint.pprint(lists)
    list_names = map(lambda x: x['name'], lists)
    assert LIST_NAME in list_names

In [38]:
setup_module(mc_api_key)

In [39]:
# lists = chimp.lists()

In [40]:
# test_ping()

In [47]:
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

try:
  client = MailchimpMarketing.Client()
  client.set_config({
    "api_key": mc_api_key,
    "server": "us1"
  })

  response = client.lists.get_all_lists()
  print(response)
except ApiClientError as error:
  print("Error: {}".format(error.text))

{'lists': [{'id': 'd7179701f9', 'web_id': 1626526, 'name': 'the_great', 'contact': {'company': 'the_great', 'address1': 'Ernst-August-Platz 1', 'address2': '', 'city': 'Hannover', 'state': 'Niedersachsen', 'zip': '30159', 'country': 'DE', 'phone': ''}, 'permission_reminder': 'You are receiving this email because you opted in via our website.', 'use_archive_bar': True, 'campaign_defaults': {'from_name': 'and', 'from_email': 'brunhalda@web.de', 'subject': '', 'language': 'en'}, 'notify_on_subscribe': '', 'notify_on_unsubscribe': '', 'date_created': '2021-04-22T18:39:14+00:00', 'list_rating': 0, 'email_type_option': False, 'subscribe_url_short': 'http://eepurl.com/hwDR2r', 'subscribe_url_long': 'https://web.us1.list-manage.com/subscribe?u=7dad146ff69eed03afd96743f&id=d7179701f9', 'beamer_address': 'us1-3cfaf80f5e-fccc9be592@inbound.mailchimp.com', 'visibility': 'prv', 'double_optin': False, 'has_welcome': False, 'marketing_permissions': False, 'modules': [], 'stats': {'member_count': 1, '

In [48]:
df_mailchimp_output.head()

,Email Address,First Name,Last Name,Address,Phone,Tags
0,mensch@mustermail.de,X,Y,Musterstraße 1 Musterort 12345 DE,,Einzelspender/in


In [108]:
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

try:
  client = MailchimpMarketing.Client()
  client.set_config({
    "api_key": mc_api_key,
    "server": "us1"
  })

  mail_adress = 'mensch4@mustermail.de'
  mail_adress_b = str.encode(mail_adress)
  mail_hash = hashlib.md5(mail_adress_b.lower())
  mail_hash_str = mail_hash.hexdigest()
    
  response = client.lists.add_list_member("d7179701f9", {"email_address": mail_adress, "status": "subscribed", "tags":["test"], "merge_fields":{"FNAME":"X"}})
  response = client.lists.set_list_member("d7179701f9", mail_hash_str, {"email_address": mail_adress, "status_if_new": "subscribed", "merge_fields":{"FNAME":"X"}})
#   response = client.lists.update_list_member("d7179701f9", mail_hash_str, {"email_address": mail_adress, "status_if_new": "subscribed", "First Name":"X"})
  print(response)
except ApiClientError as error:
  print("Error: {}".format(error.text))

{'id': '7e6aab203002f3ff6338ec32cae2945e', 'email_address': 'mensch4@mustermail.de', 'unique_email_id': '576d708c13', 'web_id': 525274990, 'email_type': 'html', 'status': 'subscribed', 'merge_fields': {'FNAME': 'X', 'LNAME': '', 'ADDRESS': '', 'PHONE': '', 'BIRTHDAY': ''}, 'stats': {'avg_open_rate': 0, 'avg_click_rate': 0}, 'ip_signup': '', 'timestamp_signup': '', 'ip_opt': '95.116.178.225', 'timestamp_opt': '2021-04-22T19:50:30+00:00', 'member_rating': 2, 'last_changed': '2021-04-22T19:57:45+00:00', 'language': '', 'vip': False, 'email_client': '', 'location': {'latitude': 0, 'longitude': 0, 'gmtoff': 0, 'dstoff': 0, 'country_code': '', 'timezone': ''}, 'source': 'API - Generic', 'tags_count': 0, 'tags': [], 'list_id': 'd7179701f9', '_links': [{'rel': 'self', 'href': 'https://us1.api.mailchimp.com/3.0/lists/d7179701f9/members/7e6aab203002f3ff6338ec32cae2945e', 'method': 'GET', 'targetSchema': 'https://us1.api.mailchimp.com/schema/3.0/Definitions/Lists/Members/Response.json'}, {'rel': 

In [82]:
mail_adress = 'mensch@mustermail.de'
mail_adress = str.encode(mail_adress)
result = hashlib.md5(mail_adress.lower())
output = result.hexdigest()
print(output)


cf9111fd0eed22845576f95e50087c04
